## Install required libraries

In [ ]:
!pip install -q bitsandbytes datasets==2.21.0 peft==0.10.0 trl==0.8.6 distilabel==1.3.2

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 527.3/527.3 kB 47.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.1/199.1 kB 20.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 245.2/245.2 kB 26.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 345.0/345.0 kB 24.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.4/59.4 MB 43.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 177.6/177.6 kB 18.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.6/132.6 kB 15.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.1/72.1 kB 7.5 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2025.3.0 requires fsspec==2025.3.0, but you have fsspec 2024.6.1 which is incompatible.


In [ ]:
pip install -U trl

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 423.1/423.1 kB 37.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 506.8/506.8 kB 47.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.7/47.7 MB 56.5 MB/s eta 0:00:00
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 18.1.0
    Uninstalling pyarrow-18.1.0:
      Successfully uninstalled pyarrow-18.1.0
  Attempting uninstall: datasets
    Found existing installation: datasets 2.21.0
    Uninstalling datasets-2.21.0:
      Successfully uninstalled datasets-2.21.0
  Attempting uninstall: trl
    Found existing installation: trl 0.8.6
    Uninstalling trl-0.8.6:
      Successfully uninstalled trl-0.8.6
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 25.6.0 requires pyarrow<20.0.0a0,>=14.0.0; platform_machine == "x86_64", but you have pyarrow 22.0.0 which is in

In [ ]:
import torch
from datasets import Dataset, load_dataset
from transformers import AutoTokenizer, AutoModelForCausalLM, TrainingArguments, BitsAndBytesConfig, GenerationConfig
from peft import LoraConfig
from trl import DPOConfig, DPOTrainer, SFTTrainer, SFTConfig
from tqdm import tqdm
import re

# SFT Training

## Format dataset for SFT

In [ ]:
# from datasets import Dataset, load_dataset
# from tqdm import tqdm

raw_data = load_dataset("Anthropic/hh-rlhf", split="train")
training_data = []
for example in tqdm(raw_data):
    conversation = re.findall(r'(?:Human|Assistant):\s*([^:]+)(?=\s*(?:Human|Assistant|$))', example["chosen"])
    messages = []
    for i, msg in enumerate(conversation):
        if i % 2 == 0:
            messages.append({"role": "user", "content": msg.strip()})
        else:
            messages.append({"role": "assistant", "content": msg.strip()})
    if messages:
        training_data.append(messages)

100%|██████████| 160800/160800 [00:08<00:00, 20040.02it/s]


In [ ]:
# Randomly sample 10k examples for training
# Use seed=42 for reproducibility
# Only the chosen column is used for SFT
import numpy as np
np.random.seed(42)

indices = np.random.choice(len(training_data), size=10000, replace=False)
training_data_10000 = [training_data[i] for i in indices]
TRAINING_DATASET = Dataset.from_dict({'messages': training_data_10000})

## Fine-tuning

In [ ]:
# Model / training params
MODEL_NAME = "Qwen/Qwen3-0.6B"
BATCH_SIZE = 8
LEARNING_RATE = 2e-4
NUM_EPOCHS = 3

In [ ]:
from getpass import getpass
huggingface_token = getpass("Enter your Huggingface token:")

Enter your Huggingface token:··········


In [ ]:
# Load tokenizer and model which is fine-tuned
# Model is loaded using 4-bit NF4 quantization
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME,
                                          token=huggingface_token
                                          )
tokenizer.pad_token = tokenizer.eos_token
model = AutoModelForCausalLM.from_pretrained(MODEL_NAME,
                                             token=huggingface_token,
                                             quantization_config=BitsAndBytesConfig(
                                                  load_in_4bit=True,
                                                  bnb_4bit_compute_dtype=torch.bfloat16,
                                                  bnb_4bit_use_double_quant=True,
                                                  bnb_4bit_quant_type='nf4'
                                              )
                                             )

# Training setup
peft_config = LoraConfig(
    lora_alpha=16,
    lora_dropout=0.1,
    target_modules=['q_proj', 'k_proj', 'v_proj', 'o_proj'],
    bias="none",
    task_type="CAUSAL_LM",
)

training_args = SFTConfig(
    output_dir='./',
    logging_steps=4,
    per_device_train_batch_size=BATCH_SIZE,
    save_only_model=True,
    learning_rate=LEARNING_RATE,
    num_train_epochs=NUM_EPOCHS,
)

sft_trainer = SFTTrainer(
    model=model,
    args=training_args,
    peft_config=peft_config,
    train_dataset=TRAINING_DATASET,
)

# Train model
sft_trainer.train()

Tokenizing train dataset:   0%|          | 0/10000 [00:00<?, ? examples/s]

Truncating train dataset:   0%|          | 0/10000 [00:00<?, ? examples/s]

The tokenizer has new PAD/BOS/EOS tokens that differ from the model config and generation config. The model config and generation config were aligned accordingly, being updated with the tokenizer's values. Updated tokens: {'bos_token_id': None, 'pad_token_id': 151643}.
/usr/local/lib/python3.12/dist-packages/torch/_dynamo/eval_frame.py:929: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)


Step,Training Loss
4,3.331000
8,2.739400
12,2.586200
16,2.436200
20,2.324800
24,2.388000
28,2.231700
32,2.294000
36,2.092300
40,2.272500


TrainOutput(global_step=3750, training_loss=1.982876802444458, metrics={'train_runtime': 2064.7116, 'train_samples_per_second': 14.53, 'train_steps_per_second': 1.816, 'total_flos': 3.4164794336477184e+16, 'train_loss': 1.982876802444458, 'entropy': 1.9048907160758972, 'num_tokens': 5512524.0, 'mean_token_accuracy': 0.567504495382309, 'epoch': 3.0})

In [ ]:
# Save model
sft_trainer.save_model("qwen3-0.6b-sft-safety")


# DPO

## Format dataset for DPO

In [ ]:
training_data_dpo = []
for example in tqdm(raw_data):
    chosen = re.findall(r'(?:Human|Assistant):\s*([^:]+)(?=\s*(?:Human|Assistant|$))', example["chosen"])
    rejected = re.findall(r'(?:Human|Assistant):\s*([^:]+)(?=\s*(?:Human|Assistant|$))', example["rejected"])
    if not chosen or not rejected:
        continue
    messages = []
    for i, msg in enumerate(chosen[:-1]):
        if i % 2 == 0:
            messages.append({"role": "user", "content": msg.strip()})
        else:
            messages.append({"role": "assistant", "content": msg.strip()})
    chosen_msg = chosen[-1].strip()
    rejected_msg = rejected[-1].strip()
    if messages and chosen_msg and rejected_msg:
        training_data_dpo.append({
            'prompt': messages,
            'chosen': chosen_msg,
            'rejected': rejected_msg
            })

100%|██████████| 160800/160800 [00:10<00:00, 14688.83it/s]


In [ ]:
# Applies chat template on prompt
def data_formulate(data, tokenizer):
    messages = data['prompt']

    # if your selected model does not have a chat template, comment out the line below and instead use: prompt = data['prompt']
    prompt = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
    return prompt

In [ ]:
# Model / training params
MODEL_NAME = "Qwen/Qwen3-0.6B"
BATCH_SIZE = 8
LEARNING_RATE = 1e-4
NUM_EPOCHS = 1

In [ ]:
import numpy as np
np.random.seed(42)

# Randomly select 10k examples for training
# Seed=42 for reproducibility
indices = np.random.choice(len(training_data_dpo), size=10000, replace=False)
training_data_dpo_10k = [training_data_dpo[i] for i in indices]

# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME,
                                          token=huggingface_token
                                          )
tokenizer.pad_token = tokenizer.eos_token

# Create dataset
prompt_list = [data_formulate(data, tokenizer) for data in training_data_dpo_10k]
chosen_list = [data['chosen'] for data in training_data_dpo_10k]
rejected_list = [data['rejected'] for data in training_data_dpo_10k]
TRAIN_DATASET_DPO = Dataset.from_dict({'prompt': prompt_list, 'chosen': chosen_list, 'rejected': rejected_list})

## Fine-Tuning

In [ ]:
# Load model using 4-bit NF4 quantization
model = AutoModelForCausalLM.from_pretrained(MODEL_NAME,
                                             token=huggingface_token,
                                             quantization_config=BitsAndBytesConfig(
                                                  load_in_4bit=True,
                                                  bnb_4bit_compute_dtype=torch.bfloat16,
                                                  bnb_4bit_use_double_quant=True,
                                                  bnb_4bit_quant_type='nf4'
                                              )
                                             )

# Training config
training_args = DPOConfig(
    output_dir='./',
    logging_steps=4,
    per_device_train_batch_size=BATCH_SIZE,
    save_only_model=True,
    learning_rate=LEARNING_RATE,
    num_train_epochs=NUM_EPOCHS,
)


peft_config = LoraConfig(
    lora_alpha=16,
    lora_dropout=0.1,
    target_modules=['q_proj', 'k_proj', 'v_proj', 'o_proj'], # Added for Qwen
    r=8,
    bias="none",
    task_type="CAUSAL_LM",
)

dpo_trainer = DPOTrainer(
    model,
    args=training_args,
    train_dataset=TRAIN_DATASET_DPO,
    processing_class=tokenizer,
    peft_config=peft_config,
)

# Run DPO
dpo_trainer.train()

Extracting prompt in train dataset:   0%|          | 0/10000 [00:00<?, ? examples/s]

Applying chat template to train dataset:   0%|          | 0/10000 [00:00<?, ? examples/s]

Tokenizing train dataset:   0%|          | 0/10000 [00:00<?, ? examples/s]

The tokenizer has new PAD/BOS/EOS tokens that differ from the model config and generation config. The model config and generation config were aligned accordingly, being updated with the tokenizer's values. Updated tokens: {'bos_token_id': None, 'pad_token_id': 151645}.
/usr/local/lib/python3.12/dist-packages/torch/utils/checkpoint.py:85: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn(


Step,Training Loss
4,0.736600
8,0.730100
12,0.704400
16,0.666900
20,0.659000
24,0.739200
28,0.718000
32,0.656700
36,0.690100
40,0.779700


TrainOutput(global_step=1250, training_loss=0.6528654159069062, metrics={'train_runtime': 1092.093, 'train_samples_per_second': 9.157, 'train_steps_per_second': 1.145, 'total_flos': 0.0, 'train_loss': 0.6528654159069062, 'epoch': 1.0})

In [ ]:
# Save model
dpo_trainer.save_model("qwen3-0.6b-dpo-safety")